In [1]:
# ref: https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/

# import
import pandas as pd
import duckdb
from mlxtend.frequent_patterns import apriori, association_rules
from googleapiclient.discovery import build
from google.oauth2 import service_account
import win32com.client
from pretty_html_table import build_table
import time

In [2]:
# time
start_time = time.time()

In [3]:
# transactions
data = pd.read_excel('Ushop L4M Data.xlsx')
data = data[['Name', 'Email', 'Financial Status', 'Fulfillment Status', 'Lineitem quantity', 'Lineitem name', 'Lineitem sku', 'Payment ID', 'Paid at']]
data.columns = ['name', 'email', 'fin_status', 'fulfil_status', 'item_qty', 'item_name', 'sku', 'pay_id', 'paid_at']
display(data)

,name,email,fin_status,fulfil_status,item_qty,item_name,sku,pay_id,paid_at
0,#43388,edwintg2300@gmail.com,paid,fulfilled,2,AXE Body Spray Musk 150ml,NaN,rUeYmuPGpuAZZbmSXyY7HbWcy,2023-04-30 23:09:25 +0600
1,#43388,edwintg2300@gmail.com,NaN,NaN,2,AXE Body Spray Excite 150ml,NaN,NaN,NaN
2,#43388,edwintg2300@gmail.com,NaN,NaN,2,AXE Body Spray Gold 150ml,NaN,NaN,NaN
3,#43388,edwintg2300@gmail.com,NaN,NaN,1,Pepsodent Toothpaste Germi-Check 200g,PEPSODENT TOOTHPASTE GERMICHECK 200G,NaN,NaN
4,#43387,minhaj.iifc@gmail.com,paid,fulfilled,4,Lifebuoy Handwash Care Refill 170ml,LB LQ CARE 170ML,rMSl8Qt4m7UJY6tcQgxXrFS6v,2023-04-30 22:57:28 +0600
...,...,...,...,...,...,...,...,...,...
27444,#38658,khaled.mohammad@unilever.com,NaN,NaN,1,Clear Shampoo Complete Active Care Anti Dandru...,CLEAR SHAMPOO COMPLETE ACTVE CARE 180ML,NaN,NaN
27445,#38658,khaled.mohammad@unilever.com,NaN,NaN,2,Sunsilk Shampoo Stunning Black Shine 170ml,SUNSILK SHAMPOO BLACK 180ML,NaN,NaN
27446,#38658,khaled.mohammad@unilever.com,NaN,NaN,1,Pepsodent Sensitive Expert Gum Care 140g,PEPSODENT TP ADV SNSTV EXPRT GUM CR 140G,NaN,NaN
27447,#38658,khaled.mohammad@unilever.com,NaN,NaN,2,Closeup Toothpaste Menthol Fresh 100g,CLOSE-UP THPSTE FRESHNSS MINT CHILL 100G,NaN,NaN


In [4]:
# clean transactions
qry = '''
select name txn_id, item_name, item_qty
from data 
where name in(select distinct name from data where fin_status='paid' and fulfil_status='fulfilled')
'''
data = duckdb.query(qry).df()
display(data)

,txn_id,item_name,item_qty
0,#43388,AXE Body Spray Musk 150ml,2
1,#43388,AXE Body Spray Excite 150ml,2
2,#43388,AXE Body Spray Gold 150ml,2
3,#43388,Pepsodent Toothpaste Germi-Check 200g,1
4,#43387,Lifebuoy Handwash Care Refill 170ml,4
...,...,...,...
25801,#38658,Clear Shampoo Complete Active Care Anti Dandru...,1
25802,#38658,Sunsilk Shampoo Stunning Black Shine 170ml,2
25803,#38658,Pepsodent Sensitive Expert Gum Care 140g,1
25804,#38658,Closeup Toothpaste Menthol Fresh 100g,2


In [5]:
# prepare transactions
basket_ushop = (data
    .groupby(['txn_id', 'item_name'])['item_qty']
    .sum().unstack().reset_index().fillna(0)
    .set_index('txn_id'))
display(basket_ushop)

item_name,AXE Body Spray Dark Temptation 150ml,AXE Body Spray Excite 150ml,AXE Body Spray Gold 150ml,AXE Body Spray Musk 150ml,Axe Signature Body Deodorant Intense 122ml,Boishakhi Gift Box,Bold & Beautiful (Gift Box),Boost Health and Nutrition Drink Jar 400g,Bru Coffee Original 50gm Buy 1 Get 1 Free,Bru Coffee Pure 200gm,...,Vim Dishwashing Liquid 100g,Vim Dishwashing Liquid 1lt,Vim Dishwashing Liquid 500ml,Vim Dishwashing Liquid Pouch 250ml,Vim Dishwashing Powder 500g,Wheel Washing Laundry Bar 100g,Wheel Washing Laundry Bar 125g,Wheel Washing Powder 2in1 Clean & Fresh 1Kg,Wheel Washing Powder 2in1 Clean & Fresh 500g,Women Plus Horlicks Health and Nutrition Drink Jar 400g
txn_id,,,,,,,,,,,,,,,,,,,,,
#38658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0
#38659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
#38660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#38661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#38662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#43384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#43385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#43386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# one-hot encode
def hot_encode(x):
    if(x <= 0): return 0
    if(x >= 1): return 1
basket_ushop = basket_ushop.applymap(hot_encode)
display(basket_ushop)

item_name,AXE Body Spray Dark Temptation 150ml,AXE Body Spray Excite 150ml,AXE Body Spray Gold 150ml,AXE Body Spray Musk 150ml,Axe Signature Body Deodorant Intense 122ml,Boishakhi Gift Box,Bold & Beautiful (Gift Box),Boost Health and Nutrition Drink Jar 400g,Bru Coffee Original 50gm Buy 1 Get 1 Free,Bru Coffee Pure 200gm,...,Vim Dishwashing Liquid 100g,Vim Dishwashing Liquid 1lt,Vim Dishwashing Liquid 500ml,Vim Dishwashing Liquid Pouch 250ml,Vim Dishwashing Powder 500g,Wheel Washing Laundry Bar 100g,Wheel Washing Laundry Bar 125g,Wheel Washing Powder 2in1 Clean & Fresh 1Kg,Wheel Washing Powder 2in1 Clean & Fresh 500g,Women Plus Horlicks Health and Nutrition Drink Jar 400g
txn_id,,,,,,,,,,,,,,,,,,,,,
#38658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
#38659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
#38660,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#38661,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#38662,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#43384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#43385,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#43386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# itemsets
frq_items_df = apriori(basket_ushop.astype('bool'), min_support=0.02, use_colnames=True)

# rules
rules_df = association_rules(frq_items_df, metric="lift", min_threshold=1)
rules_df = rules_df.sort_values(['confidence', 'lift'], ascending=[False, False])

In [8]:
# clean itemsets
frq_items_df['itemsets'] = [itemset.replace("'", "")[11:-2] for itemset in frq_items_df['itemsets'].astype('string').tolist()]
frq_items_df['items'] = [itemset.count(", ") + 1 for itemset in frq_items_df['itemsets'].tolist()] 
frq_items_df = duckdb.query('''select items, itemsets itemset, support from frq_items_df where items>1 order by support desc''').df()
display(frq_items_df)

,items,itemset,support
0,2,"Vim Dishwashing Bar 300g, Wheel Washing Powder...",0.075068
1,2,"Vim Dishwashing Bar 300g, Wheel Washing Laundr...",0.058690
2,2,"Wheel Washing Powder 2in1 Clean & Fresh 1Kg, W...",0.056187
3,2,"Vim Dishwashing Liquid 1lt, Wheel Washing Powd...",0.044586
4,2,"Vim Dishwashing Bar 300g, Vim Dishwashing Liqu...",0.036852
5,2,Rin Washing Powder Power Bright 1kg Container ...,0.036624
6,2,"Vim Dishwashing Bar 300g, Rin Washing Powder P...",0.035942
7,2,"Vim Dishwashing Liquid 1lt, Lifebuoy Handwash ...",0.035714
8,2,"Surf Excel Washing Powder with Aloe Vera 1kg, ...",0.034349
9,2,"Vim Dishwashing Bar 300g, Lifebuoy Handwash Le...",0.032985


In [9]:
# clean rules
cols = ['antecedents', 'consequents']
rules_df[cols] = rules_df[cols].applymap(lambda x: tuple(x))
rules_df = rules_df.explode('antecedents').reset_index(drop=True).explode('consequents').reset_index(drop=True)
rules_df = duckdb.query('''select antecedents antecedent, consequents consequent, support, confidence, lift, leverage, conviction, zhangs_metric from rules_df''').df()
display(rules_df)

,antecedent,consequent,support,confidence,lift,leverage,conviction,zhangs_metric
0,Knorr Soup Hot and Sour Chicken 31g,Knorr Soup Thai 28g,0.023658,0.658228,10.483948,0.021401,2.742224,0.938342
1,Knorr Soup Chicken Corn 24g,Knorr Soup Thai 28g,0.021611,0.562130,8.953349,0.019197,2.140398,0.923826
2,Lux Soap Bar Soft Glow 100g,Lux Soap Bar Velvet Glow 100g,0.022975,0.543011,10.949887,0.020877,2.079720,0.948821
3,Lux Soap Bar Soft Glow 150g,Lux Soap Bar Velvet Glow 150g,0.032985,0.510563,6.884775,0.028194,1.891648,0.913786
4,Lux Soap Bar Velvet Glow 100g,Lux Soap Bar Soft Glow 100g,0.022975,0.463303,10.949887,0.020877,1.784412,0.956088
...,...,...,...,...,...,...,...,...
105,Wheel Washing Powder 2in1 Clean & Fresh 1Kg,Pepsodent Sensitive Expert Professional 140g,0.021383,0.100967,1.349087,0.005533,1.029060,0.328283
106,Wheel Washing Powder 2in1 Clean & Fresh 1Kg,Lifebuoy Soap Bar Lemon Fresh 150g,0.020928,0.098818,1.620918,0.008017,1.042005,0.485990
107,Wheel Washing Powder 2in1 Clean & Fresh 1Kg,Lifebuoy Soap Bar Lemon Fresh 100g,0.020246,0.095596,1.765717,0.008780,1.045838,0.550176
108,Wheel Washing Powder 2in1 Clean & Fresh 1Kg,Lux Soap Bar Soft Glow 150g,0.020246,0.095596,1.479720,0.006564,1.034268,0.411304


In [10]:
# credentials
SERVICE_ACCOUNT_FILE = 'read-write-to-gsheet-apis-1-04f16c652b1e.json'
SAMPLE_SPREADSHEET_ID = '1gkLRp59RyRw4UFds0-nNQhhWOaS4VFxtJ_Hgwg2x2A0'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# APIs
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

# clear
resultClear = sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="'APRIORI'!A2:L").execute()

# update itemsets
write_df = frq_items_df.fillna('')
request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="'APRIORI'!A2", valueInputOption='USER_ENTERED', body={'values': [write_df.columns.values.tolist()] + write_df.values.tolist()}).execute()

# update rules
write_df = rules_df.fillna('')
request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range="'APRIORI'!E2", valueInputOption='USER_ENTERED', body={'values': [write_df.columns.values.tolist()] + write_df.values.tolist()}).execute()

In [11]:
# email
ol = win32com.client.Dispatch("outlook.application")
olmailitem = 0x0
newmail = ol.CreateItem(olmailitem)

# subject
newmail.Subject = 'Baskets for Ushop'

# recipients
newmail.To = 'shithi.maitra@unilever.com'
# newmail.To = 'avra.barua@unilever.com'
# newmail.CC = 'mehedi.asif@unilever.com; zakeea.husain@unilever.com; nazmussajid.ubl@gmail.com'

# body
newmail.HTMLbody = f'''
Dear concern,<br><br>
Thanks for sharing transactions from the <a href="https://ushopbd.com/">Shopify Ushopbd portal</a>. Here, I form baskets (itemsets of 'n' SKUs) using <b>APRIORI</b> algorithm. Below is a snapshot of top-05 rules:<br><br>
''' + build_table(rules_df.head(5), 'grey_dark', font_size='10px') + '''<br>
<i>A/B</i>-tests can be run on <i>Ushopbd</i> using the results. Your sharing of the method to download transactions will fully automate the process. Some definitions:<br><br>
- <b>*</b>Antecedent (<i>x</i>), <b>*</b>Consequent (<i>y</i>): If a customer buys <i>x</i>, she buys <i>y</i> too; i.e., <i>x</i> => <i>y</i><br>
- <b>*</b>Support: Frequency of <i>x</i> => <i>y</i><br>
- <b>*</b>Confidence: Probability of buying <i>y</i>, given <i>x</i> is bought<br>
- <b>*</b>Lift, <b>*</b>Leverage: Comparison of theoretical vs. observed tendencies<br>
- <b>*</b>Conviction, <b>*</b>Zhangs' metric: Correlation between <i>x</i> and <i>y</i><br><br>
You can find the complete results attached. Note, minimum support = 0.02, which may further be fine-tuned. This email is formatted and shot using <i>win32com</i>.<br><br>
Regards,<br>
Shithi Maitra<br>
Asst. Manager, Cust. Service Excellence<br>
Unilever BD Ltd.<br>
'''

# attachment
folder = r'C:\\Users\\Shithi.Maitra\\Unilever Codes\\Scraping Scripts\\'
filename = folder + "itemsets_for_ushop.xlsx"
newmail.Attachments.Add(filename)

# send
newmail.Send()

In [12]:
# stats
elapsed_time = time.time() - start_time
print("Elapsed time to report (mins): " + str(round(elapsed_time / 60.00, 2)))

Elapsed time to report (mins): 0.35
